<a href="https://colab.research.google.com/github/JSunkel/Forecasting-Models/blob/master/Line_Disconnects_Phones_Device_Group_Prophet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from fbprophet import Prophet

In [0]:
# Read a comma-separated values (csv) file into a Pandas DataFrame.
fact_table = pd.read_csv('drive/My Drive/Churn/discos_device_grp_fact_table_phone_vol_invol_dec_all.csv')

In [115]:
# Check the full dataset format (testing only)
fact_table.head()

,ds,vol_port_basic_cdma,vol_port_basic_non_cdma,vol_port_sp_android_other,vol_port_sp_apple,vol_non_port_basic_cdma,vol_non_port_basic_non_cdma,vol_non_port_sp_android_other,vol_non_port_sp_apple,invol_basic_cdma,invol_basic_non_cdma,invol_sp_android_other,invol_sp_apple
0,12/1/2016,1444,NaN,7339,7788,2926,NaN,3501,2829,411,NaN,2126,1731
1,12/2/2016,1292,NaN,8191,9137,2687,NaN,3998,2901,269,NaN,1491,1312
2,12/3/2016,1589,NaN,6589,7378,2588,NaN,2463,2261,263,NaN,1728,1370
3,12/4/2016,1690,NaN,6199,7233,2130,NaN,2196,2028,239,NaN,1226,1007
4,12/5/2016,264,NaN,1520,1646,1586,NaN,1521,1223,307,NaN,1960,1456


In [0]:
# Split out indivudal datasets for forecasting.
# Voluntary Port Outs
vol_port_basic_cdma = fact_table[['ds', 'vol_port_basic_cdma']].rename(columns = {'vol_port_basic_cdma': 'y'})
vol_port_basic_non_cdma = fact_table[['ds', 'vol_port_basic_non_cdma']].rename(columns = {'vol_port_basic_non_cdma': 'y'})
vol_port_sp_android_other = fact_table[['ds', 'vol_port_sp_android_other']].rename(columns = {'vol_port_sp_android_other': 'y'})
vol_port_sp_apple = fact_table[['ds', 'vol_port_sp_apple']].rename(columns = {'vol_port_sp_apple': 'y'})

# Voluntary non Port Outs
vol_non_port_basic_cdma = fact_table[['ds', 'vol_non_port_basic_cdma']].rename(columns = {'vol_non_port_basic_cdma': 'y'})
vol_non_port_basic_non_cdma = fact_table[['ds', 'vol_non_port_basic_non_cdma']].rename(columns = {'vol_non_port_basic_non_cdma': 'y'})
vol_non_port_sp_android_other = fact_table[['ds', 'vol_non_port_sp_android_other']].rename(columns = {'vol_non_port_sp_android_other': 'y'})
vol_non_port_sp_apple = fact_table[['ds', 'vol_non_port_sp_apple']].rename(columns = {'vol_non_port_sp_apple': 'y'})

# Involuntary
invol_basic_cdma = fact_table[['ds', 'invol_basic_cdma']].rename(columns = {'invol_basic_cdma': 'y'})
invol_basic_non_cdma = fact_table[['ds', 'invol_basic_non_cdma']].rename(columns = {'invol_basic_non_cdma': 'y'})
invol_sp_android_other = fact_table[['ds', 'invol_sp_android_other']].rename(columns = {'invol_sp_android_other': 'y'})
invol_sp_apple = fact_table[['ds', 'invol_sp_apple']].rename(columns = {'invol_sp_apple': 'y'})

In [117]:
# Check the individual dataset format (testing only)
vol_port_basic_cdma.head()

,ds,y
0,12/1/2016,1444
1,12/2/2016,1292
2,12/3/2016,1589
3,12/4/2016,1690
4,12/5/2016,264


In [0]:
# Define the holidays
jan_bill_cycle = pd.DataFrame({
    'holiday': 'jan_bill_cycle',
    'ds': pd.to_datetime(['2018-01-05',
                          '2018-01-11',
                          '2018-01-14',
                          '2018-01-17',
                          '2018-01-24',
                          '2018-01-27',
                          '2019-01-29',
                          '2018-01-30',
                          '2018-01-31',
                          '2019-01-05',
                          '2019-01-11',
                          '2019-01-14',
                          '2019-01-17',
                          '2019-01-24',
                          '2019-01-27',
                          '2019-01-29',
                          '2019-01-30',
                          '2019-01-31',
                          '2020-01-05',
                          '2020-01-11',
                          '2020-01-14',
                          '2020-01-17',
                          '2020-01-24',
                          '2020-01-27',
                          '2020-01-29',
                          '2020-01-30',
                          '2020-01-31']),
    'lower_window': 0,
    'upper_window': 0,
})

feb_bill_cycle = pd.DataFrame({
    'holiday': 'feb_bill_cycle',
    'ds': pd.to_datetime(['2018-02-05',
                          '2018-02-11',
                          '2018-02-14',
                          '2018-02-17',
                          '2018-02-24',
                          '2018-02-27',
                          '2019-02-05',
                          '2019-02-11',
                          '2019-02-14',
                          '2019-02-17',
                          '2019-02-24',
                          '2019-02-27',
                          '2020-02-05',
                          '2020-02-11',
                          '2020-02-14',
                          '2020-02-17',
                          '2020-02-24',
                          '2020-02-27',
                          '2020-02-29']),
    'lower_window': 0,
    'upper_window': 0,
})

mar_bill_cycle = pd.DataFrame({
    'holiday': 'mar_bill_cycle',
    'ds': pd.to_datetime(['2018-03-05',
                          '2018-03-11',
                          '2018-03-14',
                          '2018-03-17',
                          '2018-03-24',
                          '2018-03-27',
                          '2019-03-29',
                          '2018-03-30',
                          '2018-03-31',
                          '2019-03-05',
                          '2019-03-11',
                          '2019-03-14',
                          '2019-03-17',
                          '2019-03-24',
                          '2019-03-27',
                          '2019-03-29',
                          '2019-03-30',
                          '2019-03-31',
                          '2020-03-05',
                          '2020-03-11',
                          '2020-03-14',
                          '2020-03-17',
                          '2020-03-24',
                          '2020-03-27',
                          '2020-03-29',
                          '2020-03-30',
                          '2020-03-31']),
    'lower_window': 0,
    'upper_window': 0,
})

apr_bill_cycle = pd.DataFrame({
    'holiday': 'apr_bill_cycle',
    'ds': pd.to_datetime(['2018-04-05',
                          '2018-04-11',
                          '2018-04-14',
                          '2018-04-17',
                          '2018-04-24',
                          '2018-04-27',
                          '2019-04-29',
                          '2018-04-30',
                          '2019-04-05',
                          '2019-04-11',
                          '2019-04-14',
                          '2019-04-17',
                          '2019-04-24',
                          '2019-04-27',
                          '2019-04-29',
                          '2019-04-30',
                          '2020-04-05',
                          '2020-04-11',
                          '2020-04-14',
                          '2020-04-17',
                          '2020-04-24',
                          '2020-04-27',
                          '2020-04-29',
                          '2020-04-30']),
    'lower_window': 0,
    'upper_window': 0,
})

may_bill_cycle = pd.DataFrame({
    'holiday': 'may_bill_cycle',
    'ds': pd.to_datetime(['2018-05-05',
                          '2018-05-11',
                          '2018-05-14',
                          '2018-05-17',
                          '2018-05-24',
                          '2018-05-27',
                          '2019-05-29',
                          '2018-05-30',
                          '2018-05-31',
                          '2019-05-05',
                          '2019-05-11',
                          '2019-05-14',
                          '2019-05-17',
                          '2019-05-24',
                          '2019-05-27',
                          '2019-05-29',
                          '2019-05-30',
                          '2019-05-31',
                          '2020-05-05',
                          '2020-05-11',
                          '2020-05-14',
                          '2020-05-17',
                          '2020-05-24',
                          '2020-05-27',
                          '2020-05-29',
                          '2020-05-30',
                          '2020-05-31']),
    'lower_window': 0,
    'upper_window': 0,
})

jun_bill_cycle = pd.DataFrame({
    'holiday': 'jun_bill_cycle',
    'ds': pd.to_datetime(['2018-06-05',
                          '2018-06-11',
                          '2018-06-14',
                          '2018-06-17',
                          '2018-06-24',
                          '2018-06-27',
                          '2019-06-29',
                          '2018-06-30',
                          '2019-06-05',
                          '2019-06-11',
                          '2019-06-14',
                          '2019-06-17',
                          '2019-06-24',
                          '2019-06-27',
                          '2019-06-29',
                          '2019-06-30',
                          '2020-06-05',
                          '2020-06-11',
                          '2020-06-14',
                          '2020-06-17',
                          '2020-06-24',
                          '2020-06-27',
                          '2020-06-29',
                          '2020-06-30']),
    'lower_window': 0,
    'upper_window': 0,
})

jul_bill_cycle = pd.DataFrame({
    'holiday': 'jul_bill_cycle',
    'ds': pd.to_datetime(['2018-07-05',
                          '2018-07-11',
                          '2018-07-14',
                          '2018-07-17',
                          '2018-07-24',
                          '2018-07-27',
                          '2019-07-29',
                          '2018-07-30',
                          '2018-07-31',
                          '2019-07-05',
                          '2019-07-11',
                          '2019-07-14',
                          '2019-07-17',
                          '2019-07-24',
                          '2019-07-27',
                          '2019-07-29',
                          '2019-07-30',
                          '2019-07-31',
                          '2020-07-05',
                          '2020-07-11',
                          '2020-07-14',
                          '2020-07-17',
                          '2020-07-24',
                          '2020-07-27',
                          '2020-07-29',
                          '2020-07-30',
                          '2020-07-31']),
    'lower_window': 0,
    'upper_window': 0,
})

aug_bill_cycle = pd.DataFrame({
    'holiday': 'aug_bill_cycle',
    'ds': pd.to_datetime(['2018-08-05',
                          '2018-08-11',
                          '2018-08-14',
                          '2018-08-17',
                          '2018-08-24',
                          '2018-08-27',
                          '2019-08-29',
                          '2018-08-30',
                          '2018-08-31',
                          '2019-08-05',
                          '2019-08-11',
                          '2019-08-14',
                          '2019-08-17',
                          '2019-08-24',
                          '2019-08-27',
                          '2019-08-29',
                          '2019-08-30',
                          '2019-08-31',
                          '2020-08-05',
                          '2020-08-11',
                          '2020-08-14',
                          '2020-08-17',
                          '2020-08-24',
                          '2020-08-27',
                          '2020-08-29',
                          '2020-08-30',
                          '2020-08-31']),
    'lower_window': 0,
    'upper_window': 0,
})

sep_bill_cycle = pd.DataFrame({
    'holiday': 'sep_bill_cycle',
    'ds': pd.to_datetime(['2018-09-05',
                          '2018-09-11',
                          '2018-09-14',
                          '2018-09-17',
                          '2018-09-24',
                          '2018-09-27',
                          '2019-09-29',
                          '2018-09-30',
                          '2019-09-05',
                          '2019-09-11',
                          '2019-09-14',
                          '2019-09-17',
                          '2019-09-24',
                          '2019-09-27',
                          '2019-09-29',
                          '2019-09-30',
                          '2020-09-05',
                          '2020-09-11',
                          '2020-09-14',
                          '2020-09-17',
                          '2020-09-24',
                          '2020-09-27',
                          '2020-09-29',
                          '2020-09-30']),
    'lower_window': 0,
    'upper_window': 0,
})

oct_bill_cycle = pd.DataFrame({
    'holiday': 'oct_bill_cycle',
    'ds': pd.to_datetime(['2018-10-05',
                          '2018-10-11',
                          '2018-10-14',
                          '2018-10-17',
                          '2018-10-24',
                          '2018-10-27',
                          '2019-10-29',
                          '2018-10-30',
                          '2018-10-31',
                          '2019-10-05',
                          '2019-10-11',
                          '2019-10-14',
                          '2019-10-17',
                          '2019-10-24',
                          '2019-10-27',
                          '2019-10-29',
                          '2019-10-30',
                          '2019-10-31',
                          '2020-10-05',
                          '2020-10-11',
                          '2020-10-14',
                          '2020-10-17',
                          '2020-10-24',
                          '2020-10-27',
                          '2020-10-29',
                          '2020-10-30',
                          '2020-10-31']),
    'lower_window': 0,
    'upper_window': 0,
})

nov_bill_cycle = pd.DataFrame({
    'holiday': 'nov_bill_cycle',
    'ds': pd.to_datetime(['2018-11-05',
                          '2018-11-11',
                          '2018-11-14',
                          '2018-11-17',
                          '2018-11-24',
                          '2018-11-27',
                          '2019-11-29',
                          '2018-11-30',
                          '2019-11-05',
                          '2019-11-11',
                          '2019-11-14',
                          '2019-11-17',
                          '2019-11-24',
                          '2019-11-27',
                          '2019-11-29',
                          '2019-11-30',
                          '2020-11-05',
                          '2020-11-11',
                          '2020-11-14',
                          '2020-11-17',
                          '2020-11-24',
                          '2020-11-27',
                          '2020-11-29',
                          '2020-11-30']),
    'lower_window': 0,
    'upper_window': 0,
})

dec_bill_cycle = pd.DataFrame({
    'holiday': 'dec_bill_cycle',
    'ds': pd.to_datetime(['2018-12-05',
                          '2018-12-11',
                          '2018-12-14',
                          '2018-12-17',
                          '2018-12-24',
                          '2018-12-27',
                          '2019-12-29',
                          '2018-12-30',
                          '2018-12-31',
                          '2019-12-05',
                          '2019-12-11',
                          '2019-12-14',
                          '2019-12-17',
                          '2019-12-24',
                          '2019-12-27',
                          '2019-12-29',
                          '2019-12-30',
                          '2019-12-31',
                          '2020-12-05',
                          '2020-12-11',
                          '2020-12-14',
                          '2020-12-17',
                          '2020-12-24',
                          '2020-12-27',
                          '2020-12-29',
                          '2020-12-30',
                          '2020-12-31']),
    'lower_window': 0,
    'upper_window': 0,
})

In [0]:
holidays = pd.concat((jan_bill_cycle,
                      feb_bill_cycle,
                      mar_bill_cycle,
                      apr_bill_cycle,
                      may_bill_cycle,
                      jun_bill_cycle,
                      jul_bill_cycle,
                      aug_bill_cycle,
                      sep_bill_cycle,
                      oct_bill_cycle,
                      nov_bill_cycle,
                      dec_bill_cycle))

In [120]:
# Define and fit the models
# Voluntary Port Outs
vol_port_basic_cdma_model = Prophet(holidays = holidays).fit(vol_port_basic_cdma)
vol_port_basic_non_cdma_model = Prophet(holidays = holidays).fit(vol_port_basic_non_cdma)
vol_port_sp_android_other_model = Prophet(holidays = holidays).fit(vol_port_sp_android_other)
vol_port_sp_apple_model = Prophet(holidays = holidays).fit(vol_port_sp_apple)

# Voluntary non Port Outs
vol_non_port_basic_cdma_model = Prophet(holidays = holidays).fit(vol_non_port_basic_cdma)
vol_non_port_basic_non_cdma_model = Prophet(holidays = holidays).fit(vol_non_port_basic_non_cdma)
vol_non_port_sp_android_other_model = Prophet(holidays = holidays).fit(vol_non_port_sp_android_other)
vol_non_port_sp_apple_model = Prophet(holidays = holidays).fit(vol_non_port_sp_apple)

# Involuntary 
invol_basic_cdma_model = Prophet(holidays = holidays).fit(invol_basic_cdma)
invol_basic_non_cdma_model = Prophet(holidays = holidays).fit(invol_basic_non_cdma)
invol_sp_android_other_model = Prophet(holidays = holidays).fit(invol_sp_android_other)
invol_sp_apple_model = Prophet(holidays = holidays).fit(invol_sp_apple)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

In [0]:
# Number of periods into the future to forecast
n = 366

In [0]:
# Create dataframes to hold predictions
# Voluntary Port Outs
vol_port_basic_cdma_future = vol_port_basic_cdma_model.make_future_dataframe(periods = n)
vol_port_basic_non_cdma_future = vol_port_basic_non_cdma_model.make_future_dataframe(periods = n)
vol_port_sp_android_other_future = vol_port_sp_android_other_model.make_future_dataframe(periods = n)
vol_port_sp_apple_future = vol_port_sp_apple_model.make_future_dataframe(periods = n)

# Voluntary non Port Outs
vol_non_port_basic_cdma_future = vol_non_port_basic_cdma_model.make_future_dataframe(periods = n)
vol_non_port_basic_non_cdma_future = vol_non_port_basic_non_cdma_model.make_future_dataframe(periods = n)
vol_non_port_sp_android_other_future = vol_non_port_sp_android_other_model.make_future_dataframe(periods = n)
vol_non_port_sp_apple_future = vol_non_port_sp_apple_model.make_future_dataframe(periods = n)

# Involuntary
invol_basic_cdma_future = invol_basic_cdma_model.make_future_dataframe(periods = n)
invol_basic_non_cdma_future = invol_basic_non_cdma_model.make_future_dataframe(periods = n)
invol_sp_android_other_future = invol_sp_android_other_model.make_future_dataframe(periods = n)
invol_sp_apple_future = invol_sp_apple_model.make_future_dataframe(periods = n)

In [0]:
# Make predictions
# Voluntary Port Outs
vol_port_basic_cdma_forecast = vol_port_basic_cdma_model.predict(vol_port_basic_cdma_future)
vol_port_basic_non_cdma_forecast = vol_port_basic_non_cdma_model.predict(vol_port_basic_non_cdma_future)
vol_port_sp_android_other_forecast = vol_port_sp_android_other_model.predict(vol_port_sp_android_other_future)
vol_port_sp_apple_forecast = vol_port_sp_apple_model.predict(vol_port_sp_apple_future)

# Voluntary non Port Outs
vol_non_port_basic_cdma_forecast = vol_non_port_basic_cdma_model.predict(vol_non_port_basic_cdma_future)
vol_non_port_basic_non_cdma_forecast = vol_non_port_basic_non_cdma_model.predict(vol_non_port_basic_non_cdma_future)
vol_non_port_sp_android_other_forecast = vol_non_port_sp_android_other_model.predict(vol_non_port_sp_android_other_future)
vol_non_port_sp_apple_forecast = vol_non_port_sp_apple_model.predict(vol_non_port_sp_apple_future)

# Involuntary
invol_basic_cdma_forecast = invol_basic_cdma_model.predict(invol_basic_cdma_future)
invol_basic_non_cdma_forecast = invol_basic_non_cdma_model.predict(invol_basic_non_cdma_future)
invol_sp_android_other_forecast = invol_sp_android_other_model.predict(invol_sp_android_other_future)
invol_sp_apple_forecast = invol_sp_apple_model.predict(invol_sp_apple_future)

In [0]:
# Get the datestamp and prediction columns
# Voluntary Port Outs
vol_port_basic_cdma_forecast_subset = vol_port_basic_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Basic_CDMA'})
vol_port_basic_non_cdma_forecast_subset = vol_port_basic_non_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_Basic_non_CDMA'})
vol_port_sp_android_other_forecast_subset = vol_port_sp_android_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_SP_Android_Other'})
vol_port_sp_apple_forecast_subset = vol_port_sp_apple_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Port_SP_Apple'})

# Voluntary non Port Outs
vol_non_port_basic_cdma_forecast_subset = vol_non_port_basic_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Basic_CDMA'})
vol_non_port_basic_non_cdma_forecast_subset = vol_non_port_basic_non_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_Basic_non_CDMA'})
vol_non_port_sp_android_other_forecast_subset = vol_non_port_sp_android_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_SP_Android_Other'})
vol_non_port_sp_apple_forecast_subset = vol_non_port_sp_apple_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Vol_Non_Port_SP_Apple'})

# Involuntary
invol_basic_cdma_forecast_subset = invol_basic_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_Basic_CDMA'})
invol_basic_non_cdma_forecast_subset = invol_basic_non_cdma_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_Basic_non_CDMA'})
invol_sp_android_other_forecast_subset = invol_sp_android_other_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_SP_Android_Other'})
invol_sp_apple_forecast_subset = invol_sp_apple_forecast[['ds', 'yhat']].rename(columns = {'yhat': 'Invol_SP_Apple'})

In [125]:
# Check the format of the subsetted forecasts (testing only)
vol_port_basic_cdma_forecast_subset.tail()

,ds,Vol_Port_Basic_CDMA
1487,2020-12-27,40.928800
1488,2020-12-28,865.324644
1489,2020-12-29,87.891919
1490,2020-12-30,134.996859
1491,2020-12-31,139.405506


In [0]:
# Growth multiplier
gm = 1.00

In [0]:
# Replace any negative values and scale the forecast using the growth multiplier
# Voluntary Port Outs 
vol_port_basic_cdma_forecast_subset['Vol_Port_Basic_CDMA'] = vol_port_basic_cdma_forecast_subset['Vol_Port_Basic_CDMA'].abs() * gm
vol_port_basic_non_cdma_forecast_subset['Vol_Port_Basic_non_CDMA'] = vol_port_basic_non_cdma_forecast_subset['Vol_Port_Basic_non_CDMA'].abs() * gm
vol_port_sp_android_other_forecast_subset['Vol_Port_SP_Android_Other'] = vol_port_sp_android_other_forecast_subset['Vol_Port_SP_Android_Other'].abs() * gm
vol_port_sp_apple_forecast_subset['Vol_Port_SP_Apple'] = vol_port_sp_apple_forecast_subset['Vol_Port_SP_Apple'].abs() * gm

# Voluntary non Port Outs
vol_non_port_basic_cdma_forecast_subset['Vol_Non_Port_Basic_CDMA'] = vol_non_port_basic_cdma_forecast_subset['Vol_Non_Port_Basic_CDMA'].abs() * gm
vol_non_port_basic_non_cdma_forecast_subset['Vol_Non_Port_Basic_non_CDMA'] = vol_non_port_basic_non_cdma_forecast_subset['Vol_Non_Port_Basic_non_CDMA'].abs() * gm
vol_non_port_sp_android_other_forecast_subset['Vol_Non_Port_SP_Android_Other'] = vol_non_port_sp_android_other_forecast_subset['Vol_Non_Port_SP_Android_Other'].abs() * gm
vol_non_port_sp_apple_forecast_subset['Vol_Non_Port_SP_Apple'] = vol_non_port_sp_apple_forecast_subset['Vol_Non_Port_SP_Apple'].abs() * gm

# Involuntary
invol_basic_cdma_forecast_subset['Invol_Basic_CDMA'] = invol_basic_cdma_forecast_subset['Invol_Basic_CDMA'].abs() * gm
invol_basic_non_cdma_forecast_subset['Invol_Basic_non_CDMA'] = invol_basic_non_cdma_forecast_subset['Invol_Basic_non_CDMA'].abs() * gm
invol_sp_android_other_forecast_subset['Invol_SP_Android_Other'] = invol_sp_android_other_forecast_subset['Invol_SP_Android_Other'].abs() * gm
invol_sp_apple_forecast_subset['Invol_SP_Apple'] = invol_sp_apple_forecast_subset['Invol_SP_Apple'].abs() * gm

In [128]:
# Check the format of the adjusted forecasts (testing only)
vol_port_sp_apple_forecast_subset.tail()

,ds,Vol_Port_SP_Apple
1487,2020-12-27,3487.036144
1488,2020-12-28,9317.818248
1489,2020-12-29,3638.546520
1490,2020-12-30,3832.513109
1491,2020-12-31,3778.497935


In [0]:
# Combine the forecasts into one table
# Voluntary Port Outs
discos_device_grp_phone_pred_table = vol_port_basic_cdma_forecast_subset
discos_device_grp_phone_pred_table.insert(2, 'Vol_Port_Basic_non_CDMA', vol_port_basic_non_cdma_forecast_subset['Vol_Port_Basic_non_CDMA'])
discos_device_grp_phone_pred_table.insert(3, 'Vol_Port_SP_Android_Other', vol_port_sp_android_other_forecast_subset['Vol_Port_SP_Android_Other'])
discos_device_grp_phone_pred_table.insert(4, 'Vol_Port_SP_Apple', vol_port_sp_apple_forecast_subset['Vol_Port_SP_Apple'])

# Voluntary Non Port Outs
discos_device_grp_phone_pred_table.insert(5, 'Vol_Non_Port_Basic_CDMA', vol_non_port_basic_cdma_forecast_subset['Vol_Non_Port_Basic_CDMA'])
discos_device_grp_phone_pred_table.insert(6, 'Vol_Non_Port_Basic_non_CDMA', vol_non_port_basic_non_cdma_forecast_subset['Vol_Non_Port_Basic_non_CDMA'])
discos_device_grp_phone_pred_table.insert(7, 'Vol_Non_Port_SP_Android_Other', vol_non_port_sp_android_other_forecast_subset['Vol_Non_Port_SP_Android_Other'])
discos_device_grp_phone_pred_table.insert(8, 'Vol_Non_Port_SP_Apple', vol_non_port_sp_apple_forecast_subset['Vol_Non_Port_SP_Apple'])

# Involuntary
discos_device_grp_phone_pred_table.insert(9, 'Invol_Basic_CDMA', invol_basic_cdma_forecast_subset['Invol_Basic_CDMA'])
discos_device_grp_phone_pred_table.insert(10, 'Invol_Basic_non_CDMA', invol_basic_non_cdma_forecast_subset['Invol_Basic_non_CDMA'])
discos_device_grp_phone_pred_table.insert(11, 'Invol_SP_Android_Other', invol_sp_android_other_forecast_subset['Invol_SP_Android_Other'])
discos_device_grp_phone_pred_table.insert(12, 'Invol_SP_Apple', invol_sp_apple_forecast_subset['Invol_SP_Apple'])

In [130]:
# Check the format of the final predictions table (testing only)
discos_device_grp_phone_pred_table.tail()

,ds,Vol_Port_Basic_CDMA,Vol_Port_Basic_non_CDMA,Vol_Port_SP_Android_Other,Vol_Port_SP_Apple,Vol_Non_Port_Basic_CDMA,Vol_Non_Port_Basic_non_CDMA,Vol_Non_Port_SP_Android_Other,Vol_Non_Port_SP_Apple,Invol_Basic_CDMA,Invol_Basic_non_CDMA,Invol_SP_Android_Other,Invol_SP_Apple
1487,2020-12-27,40.928800,39.495272,1559.175777,3487.036144,285.242291,326.334189,1158.008302,2030.717058,9.231132,96.937960,1571.854469,1773.966565
1488,2020-12-28,865.324644,51.600464,5191.744078,9317.818248,947.315965,439.991192,3035.041420,4254.756032,6.595343,90.656712,1561.915235,1803.401826
1489,2020-12-29,87.891919,48.229917,1652.938609,3638.546520,334.914135,385.525178,1827.607764,2730.281466,3.336044,89.899473,1771.849326,2019.016772
1490,2020-12-30,134.996859,42.512579,1798.929038,3832.513109,346.669746,382.980215,1852.547768,2752.674611,57.318048,94.926900,2002.046892,2208.704297
1491,2020-12-31,139.405506,42.829909,1774.853670,3778.497935,333.160706,384.258512,1847.814516,2738.634517,67.530045,103.986841,1973.386993,2206.914937


In [0]:
# Save the predictions table to a csv file
discos_device_grp_phone_pred_table.to_csv('drive/My Drive/Churn/discos_device_grp_phone_pred_table.csv')